# 方差分解思想

## 单样本方差分析F检验回顾

单样本方差分析中我们分析均组间平方和MST与均组内平方和MSE的比例。在原假设成立情况下，有

$$
F = \frac{MST}{MSE} = 
\frac
{\sum_{i=1}^k n_i(\bar{x}_i - \bar{x})^2/(k-1)}
{\sum_{i=1}^k \sum_{j=1}^{n_i} (x_{ij}-\bar{x}_i)^2/(N-k)}
\sim
F(k-1,N-k)
$$

方差分析的模型假设已经将总体约束成至多均值（位置）不同的正态分布。原假设在约束均值相同后相当于直接要求所有总体同（正态）分布。
但并没有要求具体指定是哪个正态分布。

我们来证明对于所有特定分布$N(\mu,\sigma^2)$上述关系均成立。

首先各个均值统计量服从

$$
\bar{x}_i \sim N(\mu,\frac{\sigma^2}{n_i})
$$

且互相独立,令$n=\sum_{i=1}^k n_i$,整体均值则有。

$$
\bar{x} \sim N(\mu,\frac{\sigma^2}{n})
$$

整体均值当然与各分组均值不独立,

$$
cov(\bar{x}_i,\bar{x}) = cov(\bar{x}_i,\frac{1}{n}(\sum_{i=1}^k \bar{x}_i n_i)) = 
\frac{n_i}{n} cov(\bar{x}_i,\bar{x}_i) = \frac{n_i}{n}var(\bar{x}_i) = \frac{n_i}{n} \frac{\sigma^2}{n_i} = \frac{\sigma^2}{n}
$$


构造$Y_i = \bar{x}_i - \bar{x} $

$$
E(Y_i) = 0 \\
cov(Y_i,Y_j) = cov(\bar{x}_i - \bar{x},\bar{x}_j - \bar{x}) = 
cov(\bar{x},\bar{x}) - cov(\bar{x},\bar{x}_i) - cov(\bar{x},\bar{x}_j) =
-\frac{\sigma^2}{n} \quad (i \neq j) \\
cov(Y_i,Y_i) = cov(\bar{x}_i - \bar{x},\bar{x}_i - \bar{x}) = var(\bar{x}) + var(\bar{x}_i) - 2cov(\bar{x}_i,\bar{x}) = 
\frac{\sigma^2}{n_i} - \frac{\sigma^2}{n} = \sigma^2 (\frac{1}{n_i} - \frac{1}{n})
$$

再令$Z_i = \sqrt{n_i} Y_i $

$$
cov(Z_i,Z_j) = cov(\sqrt{n_i}Y_i,\sqrt{n_j}Y_j) = --\sigma^2\frac{\sqrt{n_i n_j}}{n} \\ 
cov(Z_i,Z_i) = \sigma^2 (1-\frac{n_i}{n})
$$

知道了这些，已经足够我们描述关于随机向量$Z$的多元正态分布，我们的问题是它的协方差矩阵的秩为多少。显然$Y$的协方差矩阵秩与$Z$的协方差秩相同。
所以也可以倒回去求$Y$的协方差矩阵秩。我们发现$Y$的随机分量$Y_i$间存在这么一个线性组合关系。

$$
\sum_{i=1}^k n_i Y_i = \sum_{i=1}^k n_i(\bar{x}_i - \bar{x}) = 0
$$

这意味着它的秩小于$k$(变量间的线性组合关系可以转化为协方差矩阵的行列向量的线性组合关系，这在我的某个知乎答案里讨论过)。

看上去它的秩应该就等于$k-1$,惭愧的是我没想到如何证明这一点，下面进行一些实验直接导出一些结论。

In [14]:
ni = c(10,20,30,40)
k <- length(ni)
n <- sum(ni)
sigma <- 1

Sigma.Y <- sigma^2*(diag(1/ni)-matrix(1/n,k,k))
Sigma.Y

0.09,-0.01,-0.01000000,-0.010
-0.01,0.04,-0.01000000,-0.010
-0.01,-0.01,0.02333333,-0.010
-0.01,-0.01,-0.01000000,0.015


In [23]:
ni %*% Sigma.Y 

1.665335e-16,5.551115e-17,-1.110223e-16,0


In [25]:
eigen(Sigma.Y)

0.97926997,0.0775802,-0.04096724,-0.1825742
-0.14594983,0.9085893,-0.14082185,-0.3651484
-0.10553033,-0.3534627,-0.75095103,-0.5477226
-0.09269483,-0.2085927,0.64386601,-0.7302967


In [27]:
Sigma.Z <- sqrt(diag(ni)) %*% Sigma.Y %*% t(sqrt(diag(ni)))
Sigma.Z

0.9000000,-0.1414214,-0.1732051,-0.2000000
-0.1414214,0.8000000,-0.2449490,-0.2828427
-0.1732051,-0.2449490,0.7000000,-0.3464102
-0.2000000,-0.2828427,-0.3464102,0.6000000


In [28]:
Sigma.Z %*% Sigma.Z

0.9000000,-0.1414214,-0.1732051,-0.2000000
-0.1414214,0.8000000,-0.2449490,-0.2828427
-0.1732051,-0.2449490,0.7000000,-0.3464102
-0.2000000,-0.2828427,-0.3464102,0.6000000


In [29]:
eigen(Sigma.Z)

0.9486833,0.0000000,0.00000000,-0.3162278
-0.1490712,-0.8796528,-0.06315623,-0.4472136
-0.1825742,0.3619483,-0.73188805,-0.5477226
-0.2108185,0.3085521,0.67849184,-0.6324555


我们发现，经过加权过后，$Z$的协方差矩阵“神奇的”变成了一个对称幂等矩阵。我们来证明,对于 

$$
Z \sim N(0,D_r) \\
Z^TZ \sim \chi^2(r)
$$

其中$D_r$为秩为$r$的对称幂等矩阵。在卡方分布定义原始版本里,能直接导出的只有协方差矩阵是$I_r$才有后面服从卡方分布的结论。
其中$I_r$是只有$r$个$1$元，其他$1$换成$0$的单位矩阵变动产物。$I_r$也是秩$r$的对称幂等矩阵，所以这里我们在扩展结论。

首先有结论

$$
X \sim N(0,I) \\
X^T D_r X \sim \chi^2(r)
$$

该结论可由Cochran定理导出，说实话Cochran定理本身和导出的这个结论也没那么直观。

$$
(X^T D_r^T)(D_r X) = X^T D_r X \sim \chi^2(r) \\
\Sigma(D_rX) = D_r I D_r^T = D_r
$$

也就是说，对每个协方差矩阵为对称幂等矩阵$D_r$的$0$均值多元正态分布向量$X$。我们可以反向通过$X^T D_r X \sim \chi^2(r)$得到期望结论。

从另一个角度看，因为对称幂等矩阵的特征值全为1，又因为正交矩阵乘随机向量并不改变$Z^TZ$型的分布。
我们可以直接拿它的谱分解的正交矩阵$P$做
$\Sigma(P^TZ) = P^T P P^T I_r (P^T)^T = I_r$，于是$(P^T Z)^T (P^T Z) \sim \chi^2(r) \to Z^T Z \sim \chi^2(r)$

这样，我们就得到了

$$ 
\frac{SST}{\sigma^2} = \frac{1}{\sigma^2} \sum_{i=1}^k n_i(\bar{x}_i - \bar{x})^2 = 
\sum_{i=1}^k \left(\frac{\sqrt{n_i}(\bar{x}_i - \bar{x})}{\sigma} \right)^2 =Z^TZ \sim \chi^2(k-1)
$$

也许我们可以通过类似的方法得到

$$
\frac{SSE}{\sigma^2} \sim \chi^2(n-k)
$$

(前面$k-1$的$-1$显然来自$k$个均值随机变量间的那个组合关系。这里列出$n$个随机变量后，
显然在每个对应分组随机变量内又存在一个组合关系。这种约束共$k$个，故自由度为$n-k$)

然后试图证明$SST$与$SSE$独立。再求比配个常数得到

$$
F = \frac{SST/(k-1)}{SSE/(n-k)} = \frac{MST}{MSE} \sim F(k-1,n-k)
$$

当然也可以直接选择Cochran定理证出$SSE$与$SST$的独立性与$SSE$的分布。这是我们熟悉的套路，基准是与位置参数的离差平方和，
服从自由度为$n$的卡方分布。它可以分解为三项，SST，SSE与样本均值与期望参数间的平方误差。它们显然非负定，秩关系也相对容易得到。

In [36]:
P <- eigen(Sigma.Z)$vectors
P %*%  diag(c(1,1,1,1)) %*% t(P)

1.000000e+00,-5.551115e-17,-8.326673e-17,-2.775558e-17
-5.551115e-17,1.000000e+00,-2.775558e-17,-1.110223e-16
-8.326673e-17,-2.775558e-17,1.000000e+00,2.220446e-16
-2.775558e-17,-1.110223e-16,2.220446e-16,1.000000e+00


In [41]:
A1 <- P %*% diag(c(1,1,0,0)) %*% t(P)
A2 <- P %*% diag(c(0,0,1,0)) %*% t(P)
A3 <- P %*% diag(c(0,0,0,1)) %*% t(P)

In [42]:
A1

0.9000000,-0.1414214,-0.1732051,-0.2000000
-0.1414214,0.7960113,-0.2911723,-0.2399917
-0.1732051,-0.2911723,0.1643399,0.1501699
-0.2000000,-0.2399917,0.1501699,0.1396488


In [43]:
A2

0,0.000000000,0.00000000,0.00000000
0,0.003988709,0.04622329,-0.04285099
0,0.046223290,0.53566011,-0.49658007
0,-0.042850987,-0.49658007,0.46035118


In [44]:
A3

0.1000000,0.1414214,0.1732051,0.2000000
0.1414214,0.2000000,0.2449490,0.2828427
0.1732051,0.2449490,0.3000000,0.3464102
0.2000000,0.2828427,0.3464102,0.4000000


In [45]:
A1 + A2 + A3

1.000000e+00,-5.551115e-17,-8.326673e-17,-2.775558e-17
-5.551115e-17,1.000000e+00,-2.775558e-17,-1.110223e-16
-8.326673e-17,-2.775558e-17,1.000000e+00,2.220446e-16
-2.775558e-17,-1.110223e-16,2.220446e-16,1.000000e+00


正如上面试验所指出的，

$$
P(\sum_i^k{A_i})P^T = \sum_{i=1}^k PA_iP^T = P I P^T = I
$$

其中$A_i$是除了对角线上某些元为1外其他元素皆为0的矩阵。且$\sum_i^k A_i = I$。

此法造出的$PA_iP^T$可能被构造的非常混乱，简直看不出它们依旧保持相加等于单位阵以及它们自己相对简单的秩关系以及它们作为协方差矩阵依旧拥有
的简单的平方和分布等诸多性质。

In [46]:
eigen(diag(c(0,0,1,1)))

0,0,0,1
0,0,1,0
0,1,0,0
1,0,0,0


In [47]:
eigen(A1)

0.9486833,0.0000000,0.00000000,-0.3162278
-0.1490712,-0.8796528,-0.06315623,-0.4472136
-0.1825742,0.3619483,-0.73188805,-0.5477226
-0.2108185,0.3085521,0.67849184,-0.6324555


In [48]:
eigen(A2)

0.00000000,0.00000000,1,0.0000000
-0.06315623,0.99800365,0,0.0000000
-0.73188805,-0.04631575,0,0.6798491
0.67849184,0.04293670,0,0.7333521


In [49]:
eigen(A3)

-0.3162278,0.9486833,0.00000000,0.0000000
-0.4472136,-0.1490712,0.75854107,-0.4498813
-0.5477226,-0.1825742,-0.65104666,-0.4927524
-0.6324555,-0.2108185,0.02745342,0.7448502


于是可以提出这么一个问题，是否对于
$$
\sum_{i=1}^k A_i = I \\
\sum_{i=1}^k rank(A_i) = n
$$

我们总可以找到正交矩阵$P$，使得所有$PA_iP^T$成那种单位矩阵上换几个1为0的形式。

首先，本身就是用类似上面方法生成的组合当然满足。是否除此以外根本没有其他组合？于是可以使得此定理平凡成立？

In [50]:
P

0.9486833,0.0000000,0.00000000,-0.3162278
-0.1490712,-0.8796528,-0.06315623,-0.4472136
-0.1825742,0.3619483,-0.73188805,-0.5477226
-0.2108185,0.3085521,0.67849184,-0.6324555


In [53]:
A1C <- diag(c(1,2,3,4)) %*% A1
eigen(A1C)

-0.06573833,0.7092611,0.02744780,-0.2607559
-0.68350698,-0.4506131,-0.02410082,-0.4044946
0.47256619,-0.2689469,-0.68158489,-0.8657020
0.55242907,-0.4707059,0.73082680,-0.1376607


In [55]:
eigen(P %*% A1C %*% t(P))

-0.2370583,0.8217144,-0.1912065,2.330580e-01
0.3341492,0.5181445,0.5662440,-4.709407e-05
-0.8838355,0.1620638,0.6313245,-3.018184e-01
-0.2257931,-0.1733407,-0.4942036,9.244402e-01


In [56]:
A1C

0.9000000,-0.1414214,-0.1732051,-0.2000000
-0.2828427,1.5920226,-0.5823445,-0.4799835
-0.5196152,-0.8735168,0.4930197,0.4505097
-0.8000000,-0.9599669,0.6006796,0.5585953
